# **SNOWFLAKE CORTEX SEARCH FINANCIAL SERVICES DEMO**
## Authors: John Heisler, Garrett Frere
In this demo, using [Snowflake Cortex](https://www.snowflake.com/en/data-cloud/cortex/), we will build an RAG-powered chat interface.

### RAG
We'll learn how to extract raw text from a PDF, chunk the raw text, perform prompt engineering, pass custom prompts and data to a large language model, and use Cortex Search to both handle our embeddings and retreval all without leaving Snowflake.

Specifically, we will be taking on the role of an AI Engineer who is working closely with a portfolio team at an asset manager. The portfolio team would like to deepen their comprehension of Federal Open Market Committee (FOMC) statements and meeting minutes. The FOMC determines the direction of monetary policy by directing open market operations. Ultimately the portfolio team would like an interface to ask and answer specific questions of a document wihtout searching throughout the text.

## Steps: 
1. Extract the text from our PDFs
2. Chunk the text from the PDFs
3. Create a Cortex Search Service to handle the vectorization and retreival.
4. Create a Chat interface with Streamlit in Snowflake.

**IMPORTANT:** This is meant to be a companion to the FSI_Cortex_AI_Pipeline.ipynb also in this repository. 


# 🛑 BEFORE YOU START 🛑

[Load the PDF files](https://docs.snowflake.com/en/user-guide/data-load-local-file-system-stage-ui#upload-files-onto-a-named-internal-stage) into GEN_AI_FSI.FOMC.FED_PDF_SERVER_SIDE.

In [ ]:
from snowflake.snowpark.context import get_active_session
from snowflake.core import Root
import streamlit as st
import json as json
import pandas as pd

# Step 1: Ingest PDF Text
Using the [PARSE_DOCUMENT](https://docs.snowflake.com/en/sql-reference/functions/parse_document-snowflake-cortex) function, we'll extract the text out of the PDFs in our stage. We're retaining the mardown here to later use in our [chunking strategy](https://www.snowflake.com/en/engineering-blog/impact-retrieval-chunking-finance-rag/).

In [ ]:
--create a asequence for our table
CREATE OR REPLACE SEQUENCE GEN_AI_FSI.FOMC.PDF_SEQ START = 1 INCREMENT = 1;

--create the table if it aint there. Clear it out if it is.
CREATE OR REPLACE TABLE GEN_AI_FSI.FOMC.PDF_FULL_TEXT_MARKDOWN_AWARE (
        ID INT,
        RELATIVE_PATH VARCHAR, 
        FILE_DATE DATE, 
        TEXT VARCHAR);

--insert our data into the table
INSERT INTO GEN_AI_FSI.FOMC.PDF_FULL_TEXT_MARKDOWN_AWARE (
    WITH cte AS 
    (SELECT
        GEN_AI_FSI.FOMC.PDF_SEQ.NEXTVAL as ID,
        RELATIVE_PATH,
        TRY_TO_DATE(REGEXP_SUBSTR (RELATIVE_PATH, '\\d{8}'),'YYYYMMDD') as FILE_DATE,
        SNOWFLAKE.CORTEX.PARSE_DOCUMENT(
            @GEN_AI_FSI.FOMC.FED_PDF_SERVER_SIDE,
            relative_path,
            {'mode': 'LAYOUT'}):content AS TEXT
    FROM DIRECTORY( @GEN_AI_FSI.FOMC.FED_PDF_SERVER_SIDE) ) 
    
    SELECT 
        ID, 
        RELATIVE_PATH, 
        FILE_DATE,
        trim(TEXT, '"') AS TEXT
    FROM CTE
    );

# RAG: Build the Chunk Table
Using [SPLIT_TEXT_RECURSIVE_CHARACTER](https://docs.snowflake.com/en/sql-reference/functions/split_text_recursive_character-snowflake-cortex) SQL function, we chunk the markdown PDF text into chunks that parse based on markdown, not just size and overlap. 

In [ ]:
--create the table if it aint there. Clear it out if it is.
CREATE OR REPLACE TABLE 
    GEN_AI_FSI.FOMC.PDF_CHUNKS_MARKDOWN_AWARE (
        ID INT,
        RELATIVE_PATH VARCHAR, 
        FILE_DATE DATE,
        CHUNK VARCHAR);

INSERT INTO GEN_AI_FSI.FOMC.PDF_CHUNKS_MARKDOWN_AWARE(
    SELECT
        t.ID,
        t.RELATIVE_PATH,
        t.FILE_DATE,
        CHUNKS.VALUE AS CHUNK
    FROM
        FOMC.PDF_FULL_TEXT_MARKDOWN_AWARE t ,
        LATERAL FLATTEN( input =>   SNOWFLAKE.CORTEX.SPLIT_TEXT_RECURSIVE_CHARACTER (
            text,
            'markdown',
            500, 
            150
             )) CHUNKS);

SELECT TOP 5 * FROM GEN_AI_FSI.FOMC.PDF_CHUNKS_MARKDOWN_AWARE;

# RAG: Create a Cortex Search Service

Cortex Search enables low-latency, high-quality “fuzzy” search over your Snowflake data. Cortex Search powers a broad array of search experiences for Snowflake users including Retrieval Augmented Generation (RAG) applications leveraging Large Language Models (LLMs).

We'll use this service later to power our RAG application. 

In [ ]:
--create a cortex Search Service 
CREATE OR REPLACE CORTEX SEARCH SERVICE GEN_AI_FSI.FOMC.SEARCH_FED_MARKDOWN_AWARE
    ON CHUNK
    ATTRIBUTES ID, FILE_DATE
    WAREHOUSE = FED_GEN_AI
    TARGET_LAG = '30 days'
    AS (
        SELECT 
            ID,
            FILE_DATE::string as FILE_DATE,
            CHUNK AS CHUNK  
        FROM GEN_AI_FSI.FOMC.PDF_CHUNKS_MARKDOWN_AWARE);

# RAG: FOMC Chat Interface
* We're leveraging our Cortex Search service enabling users to ask targeted questions of the documents in their stage.
* A robust chat interface could be built to handle this. For the demo, we've got a bare bones interaction.
    * You can drop this code in a stand alone streamlit in snowflake for the full expereince. 

In [ ]:
#get our session
session = get_active_session()

# access search service through Python API
root = Root(session)

search_service = (root
                  .databases["GEN_AI_FSI"]
                  .schemas["FOMC"]
                  .cortex_search_services["SEARCH_FED_MARKDOWN_AWARE"]    
)

#create a function to generate response
def complete_cs(model_name, prompt):
    cmd = f"""SELECT SNOWFLAKE.CORTEX.TRY_COMPLETE('{model_name}','{prompt}') as response"""
    df_response = session.sql(cmd).collect()
    response = df_response[0].RESPONSE
    return response


#get FOMC files
database = 'GEN_AI_FSI'
schema = 'FOMC'

#USER INPUT: select time frame
query_document_dates = f"""SELECT DISTINCT FILE_DATE FROM {database}.{schema}.PDF_CHUNKS_MARKDOWN_AWARE order by file_date desc;"""
df_document_dates = session.sql(query_document_dates).to_pandas()

#USER INPUT: select model
df_models = [
    "claude-3-5-sonnet",
    "deepseek-r1",
    "gemma-7b",
    "jamba-1.5-large",
    "jamba-1.5-mini",
    "jamba-instruct",
    "llama2-70b-chat",
    "llama3-70b",
    "llama3-8b",
    "llama3.1-405b",
    "llama3.1-70b",
    "llama3.1-8b",
    "llama3.2-1b",
    "llama3.2-3b",
    "llama3.3-70b",
    "llama4-maverick",
    "llama4-scout",
    "mistral-7b",
    "mistral-large",
    "mistral-large2",
    "reka-core",
    "reka-flash",
    "snowflake-arctic",
    "snowflake-llama-3.1-405b",
    "snowflake-llama-3.3-70b"
]

#USER INPUT: display
col1, col2 = st.columns(2)
with col1:
    user_input_date = st.selectbox("Select Document Date", df_document_dates, key="CS_date_select_box")
with col2: 
    user_input_model = st.selectbox("Select Model", df_models, key="CS_model_select_box")

#Generate a response
user_input_question = st.text_input("Ask me a question")

ask= st.button("Ask", key = "button_ask")
if ask: 
    #get the cunks that are relevant to the question
    response = search_service.search(
        user_input_question,
        columns=["ID", "FILE_DATE", "CHUNK"],
        filter={"@eq": {"FILE_DATE": f"""{user_input_date}"""} },
        limit=5,
    ).to_json()

    #st.json(response)
    # Parse the JSON5 string
    context_chunks = json.loads(response)
    
    #transform the data into a single string
    context_full = ""
    for chunk in context_chunks['results']:
        context_full += chunk['CHUNK'] + " "
    context_full = context_full.replace("'","").replace('"','')

    #build our prompt
    cs_prompt = f'''
            Role: You are an expert Senior Economist deeply knowledgeable on Federal Reserve documents and guidance including FOMC or Federal Open Market Committee 
            meeting minutes and communications. You are an expert in interpreting and answering investment-related questions based on meeting minutes and communications 
            which you are provided as context with each question.
            
            Data: You are provided with relevant text of a Federal Reserve Guidance or FOMC meeting notes relenavt to the question asked. 
            These meeting notes are generally released before the Federal Reserve takes action on economic policy.
            
            Task: Follow these instructions,
            1) Answer the question based on the context. 
            2) Be terse and do not consider information outside what you have been provided in the question and context.
            
            Output: produce thourough, valid, gramtically correct, and concise response in a professional tone. Please do not preface your response. also provide the document and location you used to answer the question.
            Context: {context_full}
            Question: Based on documents released on this date {user_input_date}, {user_input_question} 
            '''

    data = complete_cs(user_input_model, cs_prompt)
    
    with st.chat_message("model", avatar ="assistant"):
        st.write(data)